# Gerekli kütüphanelerin import'u

In [3]:
import os
import json
import cv2
import numpy as np
import face_recognition
import tensorflow as tf
import multiprocessing
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.callbacks import ModelCheckpoint

# Gerekli fonksiyonların tanımlanması

Video dosyalarını karelere ayıran fonksiyon

In [4]:
def video_to_frames(video_path):
    vidcap = cv2.VideoCapture(video_path)
    success, image = vidcap.read()
    frames = []
    while success:
        frames.append(image)
        success, image = vidcap.read()
    return frames

Karelerdeki yüzleri tespit eden ve kırpan fonksiyon

In [5]:
def detect_and_crop_faces(image):
    face_locations = face_recognition.face_locations(image)
    cropped_faces = []
    for face_location in face_locations:
        top, right, bottom, left = face_location
        face_image = image[top:bottom, left:right]
        face_image = cv2.resize(face_image, (64, 64))
        cropped_faces.append(face_image)
    return cropped_faces

JSON Dosyasını kullanarak veriyi yükleme fonksiyonu

In [6]:
def load_data_from_json(json_path, video_folder):
    with open(json_path, 'r') as file:
        data_info = json.load(file)
    
    data = []
    labels = []
    
    for video_name, info in data_info.items():
        video_path = os.path.join(video_folder, video_name)
        
        # Videoyu karelere ayır
        frames = video_to_frames(video_path)
        
        # Karelerdeki yüzleri kırp
        for frame in frames:
            cropped_faces = detect_and_crop_faces(frame)
        
            # Kırpılmış yüzleri yükle
            for face_image in cropped_faces:
                if face_image is not None:
                    data.append(face_image)
                    labels.append(1 if info['label'] == 'FAKE' else 0)
    
    data = np.array(data)
    labels = np.array(labels)
    return data, labels

# Convolutional Neural Networks(CNN) modeli

Modelin oluşturulması

In [7]:
def create_cnn_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

Model eğitme fonksiyonu

In [8]:
def train_model(json_path, video_folder):
    # Veriyi yükle
    data, labels = load_data_from_json(json_path, video_folder)

    # Veriyi eğitim ve test setlerine ayır
    X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.5, random_state=42)
    
    # Model checkpoint callback
    checkpoint_path = "model_checkpoint.h5"
    checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
     
    # Eğer checkpoint olarak .h5 dosyası var ise yükle, yoksa yeni model oluştur
    if os.path.exists(checkpoint_path):
        model = load_model(checkpoint_path)
        print("Checkpoint yüklendi.")
    else:
        model = create_cnn_model()        
        # fit fonksiyonu ile veriyi göndererek modeli eğit
        model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), callbacks=[checkpoint])
        print("Yeni model oluşturuldu.")

    return model, X_test, y_test

TensorFlow GPU kullanımını etkinleştirme

In [9]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Tüm GPU'ları kullan
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Hata durumunda yazdır
        print(e)

1 Physical GPUs, 1 Logical GPUs


Modeli Eğitme 

In [16]:
model, X_test, y_test = train_model('data/videos/metadata.json', 'data/videos')

Epoch 1/10
84/91 [==========================>...] - ETA: 0s - loss: 4.1906 - accuracy: 0.8631
Epoch 1: val_accuracy improved from -inf to 0.96914, saving model to model_checkpoint.h5
91/91 [==============================] - 3s 8ms/step - loss: 3.9159 - accuracy: 0.8696 - val_loss: 0.0819 - val_accuracy: 0.9691
Epoch 2/10
85/91 [===========================>..] - ETA: 0s - loss: 0.0481 - accuracy: 0.9864
Epoch 2: val_accuracy improved from 0.96914 to 0.97538, saving model to model_checkpoint.h5
91/91 [==============================] - 0s 5ms/step - loss: 0.0479 - accuracy: 0.9865 - val_loss: 0.0668 - val_accuracy: 0.9754
Epoch 3/10
86/91 [===========================>..] - ETA: 0s - loss: 0.0288 - accuracy: 0.9913
Epoch 3: val_accuracy improved from 0.97538 to 0.98301, saving model to model_checkpoint.h5
91/91 [==============================] - 0s 5ms/step - loss: 0.0285 - accuracy: 0.9910 - val_loss: 0.0478 - val_accuracy: 0.9830
Epoch 4/10
87/91 [===========================>..] - ETA: 0

Modeli değerlendirme

In [18]:
def evaluate_model(model, X_test, y_test):
    predictions = (model.predict(X_test) > 0.5).astype("int32")
    print(classification_report(y_test, predictions, target_names=["Real", "Fake"]))
    
evaluate_model(model, X_test, y_test)

91/91 [==============================] - 0s 3ms/step
              precision    recall  f1-score   support

        Real       1.00      1.00      1.00      1603
        Fake       1.00      0.99      0.99      1281

    accuracy                           1.00      2884
   macro avg       1.00      1.00      1.00      2884
weighted avg       1.00      1.00      1.00      2884



Spesifik video hakkında tahmin yürütme

In [22]:
# Spesifik test videoları
test_videos = ['data/specific_tests/f1.mp4','data/specific_tests/f2.mp4','data/specific_tests/f3.mp4','data/specific_tests/r1.mp4','data/specific_tests/r2.mp4']
# Eğitilmiş modelin yolu
model_path = 'model_checkpoint.h5'

def load_and_preprocess_video(video_path):
    # Videoyu karelere ayır
    frames = video_to_frames(video_path)
    
    # Karelerdeki yüzleri kırp ve preprocess et
    faces = []
    for frame in frames:
        cropped_faces = detect_and_crop_faces(frame)
        for face in cropped_faces:
            faces.append(face)
    
    # Yüzleri numpy array'e çevir
    faces = np.array(faces)
    return faces
def predict_video(video_path, model_path):
    # Videodan yüzleri yükle
    faces = load_and_preprocess_video(video_path)
    
    # Eğitilmiş modeli yükle
    model = load_model(model_path)
    
    # Yüzler üzerinde tahmin yap
    predictions = model.predict(faces)
    
    # Tahmin sonuçlarını analiz et
    fake_count = np.sum(predictions > 0.5)
    real_count = np.sum(predictions <= 0.5)
    
    if fake_count > real_count:
        print(f"Video: {video_path}, fake.")
    else:
        print(f"Video: {video_path}, gerçek.")

predict_video(test_videos[0], model_path)

10/10 [==============================] - 0s 27ms/step
Video: data/specific_tests/f1.mp4, fake.
